#### Testing parallelized combination indices generation with pycuda

This indices will form the inner loop in combinations 

In [22]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [23]:
base_len = 32
base_arr = numpy.random.randint(6, size=base_len)
cumul_arr = numpy.zeros(base_len+1)
cumul_arr[0] = 0
cumul_arr[1:] = numpy.cumsum(base_arr)
start = cumul_arr[:-2]
stop = cumul_arr[1:-1]
lengths = stop-start
pairs_lengths = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_lengths[1:] = numpy.cumsum(lengths*(lengths+1)/2)
lengths_arr = numpy.array([base_len-1]).astype(numpy.int32)
lengths = lengths.astype(numpy.int32)
start = start.astype(numpy.int32)

In [24]:
pairs_lengths.shape

(32,)

In [25]:
mod = SourceModule('''
__global__ void comb_indices(int* left,int* right,int* start,int* length,int* lengths,int* pairs_lengths)
{
    int i = threadIdx.x + blockDim.x*blockIdx.x;
    int j = threadIdx.y + blockDim.y*blockIdx.y;
    int k =  blockIdx.z*blockDim.z+threadIdx.z;
    if ( i<length[0] && j< lengths[i] && k<=j) 
    {
        left[pairs_lengths[i]+lengths[i]*(lengths[i]+1)/2-(j*(j+1)/2)-k-1] = lengths[i]-1-j + start[i];
        right[pairs_lengths[i]+lengths[i]*(lengths[i]+1)/2 - (j*(j+1)/2) -k -1] = lengths[i]-1-k + start[i];
    }
}
''')
func = mod.get_function("comb_indices")

C:\Users\Jaydeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  del sys.path[0]


In [26]:
left = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)
right = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)

In [27]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start), cuda.In(lengths_arr),
    cuda.In(lengths), cuda.In(pairs_lengths), block=(1, 16, 16), grid=(base_len-1, 1))

In [28]:
print(right)

[ 0  1  2  1  2  2  3  4  5  6  7  5  6  7  6  7  7  8  9 10  9 10 10 11 12
 13 14 15 12 13 14 15 13 14 15 14 15 15 16 17 17 18 19 20 19 20 20 21 22 23
 24 22 23 24 23 24 24 25 26 26 27 28 29 30 28 29 30 29 30 30 31 32 33 32 33
 33 34 35 35 36 37 38 39 40 41 38 39 40 41 39 40 41 40 41 41 42 43 44 43 44
 44 45 46 47 48 46 47 48 47 48 48 49 50 51 52 53 50 51 52 53 51 52 53 52 53
 53 54 55 56 55 56 56 57 58 59 58 59 59 60 61 62 63 61 62 63 62 63 63 64 65
 66 65 66 66 67 68 69 68 69 69 70 71 71 72 73 74 75 76 74 75 76 75 76 76]


In [29]:
for i in range(6):
    print("Event: {} \n Left:{} \n Right:{} \n".format(i, left[pairs_lengths[i]:pairs_lengths[i+1]], right[pairs_lengths[i]:pairs_lengths[i+1]]))

Event: 0 
 Left:[0 0 0 1 1 2] 
 Right:[0 1 2 1 2 2] 

Event: 1 
 Left:[3] 
 Right:[3] 

Event: 2 
 Left:[4 4 4 4 5 5 5 6 6 7] 
 Right:[4 5 6 7 5 6 7 6 7 7] 

Event: 3 
 Left:[ 8  8  8  9  9 10] 
 Right:[ 8  9 10  9 10 10] 

Event: 4 
 Left:[11 11 11 11 11 12 12 12 12 13 13 13 14 14 15] 
 Right:[11 12 13 14 15 12 13 14 15 13 14 15 14 15 15] 

Event: 5 
 Left:[16 16 17] 
 Right:[16 17 17] 



In [30]:
print(left)

[ 0  0  0  1  1  2  3  4  4  4  4  5  5  5  6  6  7  8  8  8  9  9 10 11 11
 11 11 11 12 12 12 12 13 13 13 14 14 15 16 16 17 18 18 18 19 19 20 21 21 21
 21 22 22 22 23 23 24 25 25 26 27 27 27 27 28 28 28 29 29 30 31 31 31 32 32
 33 34 34 35 36 37 37 37 37 37 38 38 38 38 39 39 39 40 40 41 42 42 42 43 43
 44 45 45 45 45 46 46 46 47 47 48 49 49 49 49 49 50 50 50 50 51 51 51 52 52
 53 54 54 54 55 55 56 57 57 57 58 58 59 60 60 60 60 61 61 61 62 62 63 64 64
 64 65 65 66 67 67 67 68 68 69 70 70 71 72 73 73 73 73 74 74 74 75 75 76]
